In [ ]:
!pip install transformers datasets accelerate bitsandbytes peft trl

In [ ]:
pip install datasets

In [ ]:
pip install bitsandbytes transformers peft accelerate datasets trl


In [ ]:
import transformers
print(transformers.__version__)

In [ ]:
pip install transformers==4.8.0

In [ ]:

# Step 1: Load libraries
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from datasets import load_dataset
from transformers import DataCollatorWithPadding
import torch
import bitsandbytes as bnb


In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig
)
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType

# Step 1: Load tokenizer and dataset
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

dataset = load_dataset("ag_news")
dataset = dataset.map(lambda e: tokenizer(e["text"], truncation=True, padding="max_length"), batched=True)
dataset = dataset.rename_column("label", "labels")
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Step 2: Set up QLoRA (4-bit quantization) config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# Step 3: Load quantized pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    num_labels=4
)

# Step 4: Apply LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query", "value"],  # If this fails, use ["q_proj", "v_proj"]
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_CLS
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Step 5: Training arguments
training_args = TrainingArguments(
    output_dir="./results-qlora-agnews",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    report_to="none"
)

# Step 6: Train using Trainer API
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"].select(range(2000)),  # Smaller subset for quick training
    eval_dataset=dataset["test"].select(range(500))
)

trainer.train()
